###  Impaired maturation of wild-type superoxide dismutase 1 is associated with neurodegeneration in Parkinson disease brain and a novel murine model

##### Image analysis workflow

In [ ]:
import tifffile 
from matplotlib import pyplot as plt
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import numpy as np
from skimage.morphology import remove_small_objects, dilation, erosion, ball  
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.measure import label
import cv2
#from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
#from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
import skimage.io
from skimage.measure import label, regionprops, regionprops_table
#import skimage.measure
import pandas as pd
from pathlib import Path
import glob
import seaborn as sns
import nd2
from skimage import img_as_uint
from skimage.util import invert
from vedo import Volume
import pytrax as pt
import pyvista as pv
import os

##### The folder "IMAGES" contains the imaging data. For each replicate there is the nd2 file, a coresponding "cell" mask and a "protein" mask.  

In [ ]:
os.chdir('G:\\D\\AMR_PHD_PROJECT\\OUT\\')
rootdir = Path('G:\\D\\AMR_PHD_PROJECT\\IMAGES')
file_list = [f for f in rootdir.resolve().glob('**/*') if f.is_file()]
file_list=pd.DataFrame(data=file_list)
file_list.columns = ['Data_Path']
file_list = file_list.applymap(str)
file_list['TEXT_1'] = file_list['Data_Path'].str.lstrip(r"G:\\D\AMR_PHD_PROJECT\IMAGES\Mouse\*")
file_list=file_list.assign(IMAGE_ID=file_list['TEXT_1'].str.extract('(.*)M'))
file_list['IMAGE_ID'] = file_list['IMAGE_ID'].str.replace('\\', '')
file_list['IMAGE_ID'] = "MS" + file_list['IMAGE_ID'] 
file_list=file_list.dropna()
UNIQUE_ID =np.unique(file_list['IMAGE_ID'].astype(str))
out='G:\\D\\AMR_PHD_PROJECT\\OUT\\'
ref_df = pd.read_csv('G:\\D\\AMR_PHD_PROJECT\\sod1_mouse_ref.csv')
#141-ROI3 eg of image name
ref_df['ID'] = ref_df['Folder name'].astype(str)
ref_df['IMAGE_ID'] = ref_df['Folder name']

##### Create empty folders to hold information as the loop runs

In [ ]:
Cell_Volume = []
Void_Volume = []
Protein_in_Cell_Volume = []
Protein_in_Void_Volume = []

##### Run the image processing steps

In [ ]:
for ID in UNIQUE_ID:
    ID_BOOL=file_list.IMAGE_ID ==  ID
    SUB_DF = file_list[ID_BOOL]
    ID_TXT= ID
    
    ID_BOOL_ref=ref_df.IMAGE_ID ==  ID
    ref_sub = ref_df[ID_BOOL_ref]
    Genotype_id=ref_sub['Genotype']
    Genotype_id_string=str(np.array(Genotype_id))
    
    #Read in Cells
    cell_df = SUB_DF[SUB_DF['TEXT_1'].str.contains("cells")]
    cell_path=(cell_df.Data_Path)
    cell_path=np.array(cell_path)
    
    cell_img=tifffile.imread(cell_path[0])
    print("Images from folder")
    print(cell_df.IMAGE_ID)
    print("shape of image Z,X, Y:" )
    print(cell_img.shape)
    print("Number of unique cells:")
    print(np.unique(cell_img))
    
    nd2_df=SUB_DF[SUB_DF['TEXT_1'].str.contains("nd2")]
    nd2_path=(nd2_df.Data_Path)
    nd2_path=np.array(nd2_path)
    nd2_img=nd2.ND2File(nd2_path[0])
    
    #extract voxel size from nd2 raw file
    voxel_size=nd2_img.voxel_size()
    X_VOX_SIZE =voxel_size[0]
    Y_VOX_SIZE =voxel_size[1]
    Z_VOX_SIZE =voxel_size[2]
    print(nd2_img.voxel_size())
    
    #Read in protein 
    protein_df = SUB_DF[SUB_DF['TEXT_1'].str.contains("protein")]
    protein_path=(protein_df.Data_Path)
    protein_path=np.array(protein_path)
    protein=tifffile.imread(protein_path[0])

    #Cells are segmented indivdually, here we make them binary
    cells_binary=cell_img.copy()
    cells_binary[cells_binary !=0 ] = 1
    
    #Void is any "0" pixel in the cell 3D stack
    void_binary=cell_img.copy()
    void_binary = (void_binary == 0).astype(np.uint8)
    
    # using "*" essentially locks the object to its source, so only the protein in cells or void below
    protein_in_cell = cells_binary * protein
    protein_in_void = void_binary * protein
    
    #Create STL's using Vedo and Plot using Pyvista
    CELL_VOL = Volume(cells_binary, spacing=(Z_VOX_SIZE,Y_VOX_SIZE,X_VOX_SIZE))
    CELL_ISO = CELL_VOL.isosurface(1)
    CELL_ISO.write(ID+"CELLS.stl")
    CELL_VOL_STL=pv.read(ID+"CELLS.stl")
    
    PROTEIN_VOL = Volume(protein, spacing=(Z_VOX_SIZE,Y_VOX_SIZE,X_VOX_SIZE))
    PROTEIN_ISO =  PROTEIN_VOL.isosurface(1)
    PROTEIN_ISO.write(ID+"ALL_PROTEIN.stl")
    PROTEIN_VOL_STL=pv.read(ID+"ALL_PROTEIN.stl")

    
    CELL_PROTEIN_VOL = Volume(protein_in_cell, spacing=(Z_VOX_SIZE,Y_VOX_SIZE,X_VOX_SIZE))
    CELL_PROTEIN_ISO = CELL_PROTEIN_VOL.isosurface(1)
    CELL_PROTEIN_ISO.write(ID+"CELL_PROTEIN.stl")
    CELL_PROTEIN_STL=pv.read(ID+"CELL_PROTEIN.stl")
    
    VOID_PROTEIN_VOL = Volume(protein_in_void, spacing=(Z_VOX_SIZE,Y_VOX_SIZE,X_VOX_SIZE))
    VOID_PROTEIN_ISO = VOID_PROTEIN_VOL.isosurface(1)
    VOID_PROTEIN_ISO.write(ID+"VOID_PROTEIN.stl")
    VOID_PROTEIN_STL=pv.read(ID+"VOID_PROTEIN.stl")
    

    #Need an if else statement for when there is no protein 
    if PROTEIN_VOL_STL.n_points != 0:
        pv.set_plot_theme("document")
        p = pv.Plotter()
        p.add_mesh(CELL_VOL_STL, color="grey", opacity=0.4).rotate_y(90);
        p.add_title(Genotype_id_string)
        p.add_mesh(PROTEIN_VOL_STL, color="red", opacity=1).rotate_y(90);
        p.window_size = 500, 500
        p.show(jupyter_backend='static');
        p.screenshot(ID+Genotype_id_string+'3D_VIS.png') ;
        
    else:
        pv.set_plot_theme("document")
        p = pv.Plotter()
        p.add_mesh(CELL_VOL_STL, color="grey", opacity=0.4).rotate_y(90);
        p.add_title(Genotype_id_string)
        p.window_size = 500, 500
        p.show(jupyter_backend='static')
        p.screenshot(ID+Genotype_id_string+'3D_VIS.png') ;
    

    #use regions props to extract volume (um^3)
    Protein_Labels = label((protein_in_cell))
    PROTEIN_INFO = skimage.measure.regionprops_table(Protein_Labels, intensity_image=cell_img,  properties = ('label','area', 'intensity_max'))
    PROTEIN_INFO=pd.DataFrame(PROTEIN_INFO)
    PROTEIN_INFO.columns = ['Protein_Label', 'Protein_Area_Pixels', "Cell_Label"]
    PROTEIN_INFO['Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO['Protein_Area_Pixels'] * X_VOX_SIZE *Y_VOX_SIZE*Z_VOX_SIZE
    PROTEIN_INFO["ID"] = ID
    Protein_in_Cell_Volume.append(PROTEIN_INFO)
    print("Protein Volume Inside Cells'")
    print(PROTEIN_INFO['Proten_Volume_Voxel_Resolved'])
    

    Protein_Labels_void = label((protein_in_void))
    PROTEIN_INFO_void = skimage.measure.regionprops_table(Protein_Labels_void, properties = ('label','area'))
    PROTEIN_INFO_void=pd.DataFrame(PROTEIN_INFO_void)
    PROTEIN_INFO_void.columns = ['Protein_Label', 'Protein_Area_Pixels']
    PROTEIN_INFO_void['Proten_Volume_Voxel_Resolved'] = PROTEIN_INFO_void['Protein_Area_Pixels'] * X_VOX_SIZE *Y_VOX_SIZE*Z_VOX_SIZE
    PROTEIN_INFO_void["ID"] = ID
    Protein_in_Void_Volume.append(PROTEIN_INFO_void)
    print("Protein Volume Inside Void'")
    print(PROTEIN_INFO_void['Proten_Volume_Voxel_Resolved'])
    
    
    CELL_INFO = skimage.measure.regionprops_table(cell_img,  properties = ('label','area'))
    CELL_INFO=pd.DataFrame(CELL_INFO)
    CELL_INFO.columns = ['Cell_Label', 'Cell_Area_Pixels', ]
    CELL_INFO['Cell_Volume_Voxel_Resolved'] = CELL_INFO['Cell_Area_Pixels'] * X_VOX_SIZE *Y_VOX_SIZE*Z_VOX_SIZE
    CELL_INFO["ID"] = ID
    Cell_Volume.append(CELL_INFO)
    
    print("Cell Volume")
    print(CELL_INFO['Cell_Volume_Voxel_Resolved'])
    
    VOID_INFO = skimage.measure.regionprops_table(void_binary,  properties = ('label','area'))
    VOID_INFO=pd.DataFrame(VOID_INFO)
    VOID_INFO.columns = ['Void_Label', 'Void_Area_Pixels', ]
    VOID_INFO['Void_Volume_Voxel_Resolved'] = VOID_INFO['Void_Area_Pixels'] * X_VOX_SIZE *Y_VOX_SIZE*Z_VOX_SIZE
    VOID_INFO["ID"] = ID
    Void_Volume.append(VOID_INFO)
    
    print("Void Volume")
    print(VOID_INFO['Void_Volume_Voxel_Resolved'])
    
    
    print("Move On to Next Image")

print("No more images left")    
    
    